In [ ]:
# MODEL = 'efficientdet-d0' \
# import os \
# def download(m):\
#   if m not in os.listdir():\
#     !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{m}.tar.gz \
#     !tar zxf {m}.tar.gz \
#   ckpt_path = os.path.join(os.getcwd(), m) \
#   return ckpt_path

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import wandb

In [11]:
import package as pk

In [21]:
model = 'ResNet50'
shape = 224
batch = 1
run = wandb.init(project="test_project", name=f'{model} {shape}x{shape} {batch}')

_runtime,250
_timestamp,1627469646
_step,2
spineNet_inference_time,292.21851
spineNet_throughput_time,292.21851
spineNet_std,84.78673
inference_time,292.21851
throughput_time,292.21851
std,84.78673


_runtime,▁▄█
_timestamp,▁▄█
_step,▁▅█
spineNet_inference_time,▁
spineNet_throughput_time,▁
spineNet_std,▁
inference_time,▁
throughput_time,▁
std,▁


In [22]:
benchmarker = pk.utils.Benchmark(model, batch, (shape,shape), device='CPU:0')
benchmarks = benchmarker.execute()

In [24]:
columns = ["Benchmark", "Time (milliseconds)"]

In [25]:
table = wandb.Table(data=[[x,y] for (x, y) in benchmarks.items()], columns=columns)

In [26]:
wandb.log({"infer_benchmarks" : wandb.plot.bar(table, *columns,
           title="Inference Benchmarks")})

In [27]:
wandb.log(benchmarks)

In [ ]:
# run.join()